In [1]:
import json

In [2]:
PLAN_DEFAULT = {
    "object_type": "chair",
    "object_dims": {"width": 0.5, "depth": 0.5},
    "target": {"name": None},
    "weights": {"near_target": 0.2, "max_clearance": 0.7, "near_wall": 0.1},
    "constraints": {"min_clearance": 0.10, "boundary_margin": 0.10},
    "wall_pref": "neutral"
}

In [3]:
def normalize_weights(w):
    w = {k: max(0.0, float(v)) for k, v in w.items()}
    s = sum(w.values())
    if s <= 0:
        return PLAN_DEFAULT["weights"].copy()
    return {k: v/s for k, v in w.items()}

In [4]:
def validate_plan(plan, available_targets):
    out = PLAN_DEFAULT.copy()

    # object dims
    dims = plan.get("object_dims", {})
    out["object_dims"] = {
        "width": float(dims.get("width", out["object_dims"]["width"])),
        "depth": float(dims.get("depth", out["object_dims"]["depth"]))
    }

    # target
    tname = plan.get("target", {}).get("name", None)
    out["target"] = {"name": tname if tname in available_targets else None}

    # weights
    out["weights"] = normalize_weights(plan.get("weights", out["weights"]))

    # constraints
    c = plan.get("constraints", {})
    out["constraints"] = {
        "min_clearance": float(c.get("min_clearance", out["constraints"]["min_clearance"])),
        "boundary_margin": float(c.get("boundary_margin", out["constraints"]["boundary_margin"]))
    }

    # wall preference
    wp = plan.get("wall_pref", "neutral")
    out["wall_pref"] = wp if wp in ["near","far","neutral"] else "neutral"

    return out